In [49]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from datetime import datetime

from torch.utils.tensorboard import SummaryWriter
# Créer un writer avec timestamp pour identifier les runs
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter(f'runs/training_{timestamp}')
print(f"📊 TensorBoard logs: runs/training_{timestamp}")

# Configuration du device (GPU si dispo, sinon CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Utilisation de : {device}")

# Transformations : Conversion en Tensor + Normalisation
# On ajoute une fonction lambda pour décaler les labels de 1 (1-26 -> 0-25)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    transforms.RandomHorizontalFlip(1),
    transforms.RandomRotation(90)
     # Normalisation moyenne 0.5, std 0.5
])

# Fonction pour corriger les labels (target - 1)
def target_transform(target):
    return target - 1

# Téléchargement du dataset EMNIST (Lettres)
train_dataset = torchvision.datasets.EMNIST(
    root='./data', 
    split='letters', 
    train=True, 
    download=True, 
    transform=transform,
    target_transform=target_transform
)

test_dataset = torchvision.datasets.EMNIST(
    root='./data', 
    split='letters', 
    train=False, 
    download=True, 
    transform=transform,
    target_transform=target_transform
)

# Création des DataLoaders
batch_size = 64
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

📊 TensorBoard logs: runs/training_20251211_164459
Utilisation de : cpu


In [50]:
class CNN_MLP_Network(nn.Module):
    def __init__(self):
        super(CNN_MLP_Network, self).__init__()
        
        # --- PARTIE 1 : Extraction de Features (CNN) ---
        self.cnn_layers = nn.Sequential(
            # Conv Layer 1 : Input 1 canal (gris), 32 filtres
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # Image devient 14x14
            
            # Conv Layer 2 : Input 32, 64 filtres
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # Image devient 7x7
        )
        
        # --- PARTIE 2 : Classification (MLP) ---
        self.mlp_layers = nn.Sequential(
            nn.Flatten(), # Aplatit les données (64 * 7 * 7)
            
            # Couche cachée du MLP (Dense layer)
            nn.Linear(64 * 7 * 7, 512), 
            nn.ReLU(),
            nn.Dropout(0.5), # Pour éviter le surapprentissage
            
            # Couche de sortie (26 lettres)
            nn.Linear(512, 26) 
        )

    def forward(self, x):
        # Passage dans les filtres CNN
        x = self.cnn_layers(x)
        # Passage dans le MLP
        x = self.mlp_layers(x)
        return x

# Initialisation du modèle

model = CNN_MLP_Network().to(torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu")

In [51]:
# Hyperparamètres
learning_rate = 0.001
num_epochs = 5 # Tu peux augmenter ce nombre pour de meilleurs résultats

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

print("Début de l'entraînement...\n")

for epoch in range(num_epochs):
    # --- ENTRAÎNEMENT ---
    model.train()
    running_loss = 0.0
    
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass et optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if (i+1) % 100 == 0:
            avg_loss = running_loss / 100
            writer.add_scalar('training loss', avg_loss, epoch * len(train_loader) + i)
            running_loss = 0.0
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
    
    # --- ÉVALUATION À LA FIN DE CHAQUE EPOCH ---
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    writer.add_scalar('test accuracy', accuracy, epoch)
    print(f'Epoch [{epoch+1}/{num_epochs}] → Test Accuracy: {accuracy:.2f}% ({correct}/{total})\n')

print("✅ Entraînement terminé !")
writer.close()
print("📊 TensorBoard logs sauvegardés")

Début de l'entraînement...

Epoch [1/5], Step [100/1950], Loss: 2.5247
Epoch [1/5], Step [100/1950], Loss: 2.5247
Epoch [1/5], Step [200/1950], Loss: 1.8078
Epoch [1/5], Step [200/1950], Loss: 1.8078
Epoch [1/5], Step [300/1950], Loss: 1.2510
Epoch [1/5], Step [300/1950], Loss: 1.2510
Epoch [1/5], Step [400/1950], Loss: 1.3366
Epoch [1/5], Step [400/1950], Loss: 1.3366
Epoch [1/5], Step [500/1950], Loss: 1.3538
Epoch [1/5], Step [500/1950], Loss: 1.3538
Epoch [1/5], Step [600/1950], Loss: 1.3053
Epoch [1/5], Step [600/1950], Loss: 1.3053
Epoch [1/5], Step [700/1950], Loss: 0.9797
Epoch [1/5], Step [700/1950], Loss: 0.9797
Epoch [1/5], Step [800/1950], Loss: 1.3639
Epoch [1/5], Step [800/1950], Loss: 1.3639
Epoch [1/5], Step [900/1950], Loss: 1.2051
Epoch [1/5], Step [900/1950], Loss: 1.2051
Epoch [1/5], Step [1000/1950], Loss: 0.9854
Epoch [1/5], Step [1000/1950], Loss: 0.9854
Epoch [1/5], Step [1100/1950], Loss: 1.0535
Epoch [1/5], Step [1100/1950], Loss: 1.0535
Epoch [1/5], Step [120

In [52]:
import onnx
import os

# 🗑️ Nettoyer les anciens fichiers .data
for file in os.listdir("."):
    if file.endswith(".onnx.data"):
        os.remove(file)
        print(f"Supprimé : {file}")

model.eval()
dummy_input = torch.randn(1, 1, 28, 28)

print("🔧 Export ONNX avec tous les poids en dur...")

torch.onnx.export(
    model,
    dummy_input,
    "mon_modele_v2.onnx",
    export_params=True,
    opset_version=18,  # Version plus récente et compatible
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
)

print("✅ Fichier créé. Optimisation en cours...")


onnx_model = onnx.load("mon_modele_v2.onnx")
onnx.checker.check_model(onnx_model)


for initializer in onnx_model.graph.initializer:
    if initializer.HasField("raw_data"):
        continue  # Déjà en dur
    
onnx.save_model(onnx_model, "mon_modele_v2.onnx", save_as_external_data=False)

print("✅ 'mon_modele_v2.onnx' optimisé et sauvegardé!")
print("   ✓ Tous les poids intégrés dans le fichier unique")
print("   ✓ Aucun fichier .data externe")

onnx_data_files = [f for f in os.listdir(".") if f.endswith(".onnx.data")]
if onnx_data_files:
    print(f"⚠️ Fichiers .data restants : {onnx_data_files}")
else:
    print("✓ Pas de fichiers .data externes")

Supprimé : mon_modele_v2.onnx.data
🔧 Export ONNX avec tous les poids en dur...
[torch.onnx] Obtain model graph for `CNN_MLP_Network([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `CNN_MLP_Network([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Obtain model graph for `CNN_MLP_Network([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `CNN_MLP_Network([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
✅ Fichier créé. Optimisation en cours...
✅ 'mon_modele_v2.onnx' optimisé et sauvegardé!
   ✓ Tous les poids intégrés dans le fichier unique
   ✓ Aucun fichier .data externe
⚠️ Fichiers .data restants : ['mon_modele_v2.onnx.data']
[torch.onnx] Run decomposition... ✅
[torch.onnx] Tran

In [27]:
train_dataset.data[1]

tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
           8,  32,  37,  37,  37,  37,  37,  20,   7,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   4,  22,
          46, 114, 127, 127, 127, 127, 125,  77,  32,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   1,  22,  46, 115, 172,
         208, 245, 250, 250, 250, 250, 249, 206, 126,   8,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   3,  36, 159, 207, 245, 252,
         254, 254, 254, 255, 255, 254, 254, 245, 204,  34,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   